<a href="https://colab.research.google.com/github/kmongsil1105/colab_ipynb/blob/main/AI_PyTorch(nn_Module%EB%A1%9C_%EA%B5%AC%ED%98%84_%EC%84%A0%ED%98%95%ED%9A%8C%EA%B7%80).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# nn.Module로 구현하는 선형 회귀

 파이토치에서는 선형 회귀 모델이 nn.Linear()라는 함수로, 
 
 또 평균 제곱오차가 nn.functional.mse_loss()라는 함수로 구현되어 있다.

1. 단순 선형 회귀 구현하기

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
torch.manual_seed(1)

   데이터 선언

In [ ]:
# 데이터
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[2], [4], [6]])

선형 회귀 모델을 구현 ==> nn.Linear()는 입력의 차원, 출력의 차원을 인수로 받습니다.

In [ ]:
# 모델을 선언 및 초기화. 단순 선형 회귀이므로 input_dim=1, output_dim=1.
model = nn.Linear(1,1)

하나의 입력 에 대해서 하나의 출력 을 가지므로, 

입력 차원과 출력 차원 모두 1을 인수로 사용하였습니다. 

model에는 가중치 W와 편향 b가 저장되어져 있습니다. 

이 값은 model.parameters()라는 함수를 사용하여 불러올 수 있는데, 한 번 출력해보겠습니다.

In [ ]:
print(list(model.parameters()))

[Parameter containing:
tensor([[-0.2816]], requires_grad=True), Parameter containing:
tensor([-0.1690], requires_grad=True)]


2개의 값이 출력되는데 "첫번째 값이 W"고, "두번째 값이 b"에 해당됩니다. 

두 값 모두 현재는 랜덤 초기화가 되어져 있습니다. 

그리고 두 값 모두 학습의 대상이므로 requires_grad=True가 되어져 있는 것을 볼 수 있습니다

-------------------

이제 옵티마이저를 정의합니다. model.parameters()를 사용하여 W와 b를 전달합니다.


학습률(learning rate)은 0.01로 정합니다.

In [ ]:
# optimizer 설정. 경사 하강법 SGD를 사용하고 learning rate를 의미하는 lr은 0.01

optimizer = torch.optim.SGD(model.parameters(), lr=0.01) 

In [ ]:
# 전체 훈련 데이터에 대해 경사 하강법을 2,000회 반복
nb_epochs = 2000
for epoch in range(nb_epochs+1):

    # H(x) 계산
    prediction = model(x_train)

    # cost 계산
    cost = F.mse_loss(prediction, y_train) # <== 파이토치에서 제공하는 평균 제곱 오차 함수

    # cost로 H(x) 개선하는 부분
    # gradient를 0으로 초기화
    optimizer.zero_grad()
    # 비용 함수를 미분하여 gradient 계산
    cost.backward() # backward 연산
    # W와 b를 업데이트
    optimizer.step()

    if epoch % 100 == 0:
    # 100번마다 로그 출력
      print('Epoch {:4d}/{} Cost: {:.6f}'.format(
          epoch, nb_epochs, cost.item()
      ))

Epoch    0/2000 Cost: 25.865189
Epoch  100/2000 Cost: 0.043357
Epoch  200/2000 Cost: 0.026792
Epoch  300/2000 Cost: 0.016556
Epoch  400/2000 Cost: 0.010231
Epoch  500/2000 Cost: 0.006322
Epoch  600/2000 Cost: 0.003907
Epoch  700/2000 Cost: 0.002414
Epoch  800/2000 Cost: 0.001492
Epoch  900/2000 Cost: 0.000922
Epoch 1000/2000 Cost: 0.000570
Epoch 1100/2000 Cost: 0.000352
Epoch 1200/2000 Cost: 0.000217
Epoch 1300/2000 Cost: 0.000134
Epoch 1400/2000 Cost: 0.000083
Epoch 1500/2000 Cost: 0.000051
Epoch 1600/2000 Cost: 0.000032
Epoch 1700/2000 Cost: 0.000020
Epoch 1800/2000 Cost: 0.000012
Epoch 1900/2000 Cost: 0.000007
Epoch 2000/2000 Cost: 0.000005


 * 학습이 완료되었습니다. 
 
  Cost의 값이 매우 작습니다. 
  
  W와 b의 값도 최적화가 되었는지 확인해봅시다.

  x에 임의의 값 4를 넣어 모델이 예측하는 y의 값을 확인해보겠습니다.

In [ ]:
# 임의의 입력 4를 선언
new_var =  torch.FloatTensor([[4.0]]) 
# 입력한 값 4에 대해서 예측값 y를 리턴받아서 pred_y에 저장
pred_y = model(new_var) # forward 연산
# y = 2x 이므로 입력이 4라면 y가 8에 가까운 값이 나와야 제대로 학습이 된 것
print("훈련 후 입력이 4일 때의 예측값 :", pred_y) 

훈련 후 입력이 4일 때의 예측값 : tensor([[7.9957]], grad_fn=<AddmmBackward>)


 * 사실 이 문제의 정답은 y = 2x가 정답이므로 
 
   y값이 8에 가까우면 W와 b의 값이 어느정도 최적화가 된 것으로 볼 수 있습니다. 
   
   실제로 예측된 y값은 8에 매우 가깝습니다.

 * 이제 학습 후의 W와 b의 값을 출력해보겠습니다.

In [ ]:
print(list(model.parameters()))

[Parameter containing:
tensor([[1.9975]], requires_grad=True), Parameter containing:
tensor([0.0057], requires_grad=True)]


 * W의 값이 2에 가깝고, b의 값이 0에 가까운 것을 볼 수 있습니다.

 ---------------------------------------------------------------------

 * H(x)식에 입력 x로부터 예측된 y를 얻는 것을 forward 연산이라고 합니다.

 *학습 전, prediction = model(x_train)은 x_train으로부터 "예측값을 리턴"하므로 "forward 연산"입니다.

 *학습 후, pred_y = model(new_var)는 임의의 값 new_var로부터 "예측값을 리턴"하므로 "forward 연산"입니다.

 *학습 과정에서 "비용 함수를 미분하여 기울기를 구하는 것"을 "backward 연산"이라고 합니다.

 *cost.backward()는 비용 함수로부터 기울기를 구하라는 의미이며  "backward 연산"입니다.